In [1]:
import numpy as np
from qutip import basis, ket2dm, tensor, expect, fidelity, qeye

# Define the states
H = basis(2,0)
V = basis(2,1)

## Maximally entangled EPR pairs:
phi_plus_ket = (tensor(H, H) + tensor(V, V)).unit()
phi_plus = ket2dm(phi_plus_ket)

## Maximally entangled EPR pairs with mixture:
rho = 0.7 * phi_plus + 0.15 * ket2dm(tensor(H, V)) + 0.15 * ket2dm(tensor(V, H))


## Non-maximally entangled pair:
phi_ket = (1/np.sqrt(5)*tensor(H, H) + 2/np.sqrt(5)*tensor(V, V)).unit()
phi = ket2dm(phi_ket)


## Non-maximally entangled pair with mixture:
rho_prime = 0.7*phi + 0.15*ket2dm(tensor(H, V)) + 0.15*ket2dm(tensor(V, H))

all_states = {
    "Phi+": phi_plus,
    "rho": rho,
    "phi": phi,
    "rho_prime": rho_prime
}

Matplotlib is building the font cache; this may take a moment.


### Generate entangled-pair timestamps (shared for Alice & Bob)

In [2]:
def generate_pair_timestamps(pair_rate, total_time):
    """
    Generate an array of arrival times in [0, total_time],
    from a Poisson process with average rate = pair_rate.
    """
    n_pairs = np.random.poisson(pair_rate * total_time)
    return np.random.rand(n_pairs) * total_time

### Detect real photons on each side from the shared pair times

In [3]:
def detect_photons_from_pairs(pair_times, eff_detector, dead_time):
    times_sorted = np.sort(pair_times)
    detect_times = []
    last_det_t = -np.inf
    for t in times_sorted:
        if t - last_det_t < dead_time:
            continue
        # attempt detection
        if np.random.rand() < eff_detector:
            detect_times.append(t)
            last_det_t = t
    return np.array(detect_times)

### Generate & detect dark counts

In [4]:
def generate_dark_counts(dark_rate, eff_detector, dead_time, total_time):
    n_dark = np.random.poisson(dark_rate * total_time)
    dark_arrivals = np.sort(np.random.rand(n_dark) * total_time)

    detect_times = []
    last_det_t = -np.inf
    for t in dark_arrivals:
        if t - last_det_t < dead_time:
            continue
        # attempt detection
        if np.random.rand() < eff_detector:
            detect_times.append(t)
            last_det_t = t
    return np.array(detect_times)

### Counting coincidences with a 1 ns window, separating photon/dark

In [5]:
def count_coincidences(a_photon, a_dark, b_photon, b_dark, coinc_window=1e-9):
    """
    Return a dict with number of coincidences in each category:
      - photon-photon
      - photon-dark
      - dark-photon
      - dark-dark
    """
    cdict = {}
    cdict["photon-photon"] = _count_coincidences_2arr(a_photon, b_photon, coinc_window)
    cdict["photon-dark"]   = _count_coincidences_2arr(a_photon, b_dark, coinc_window)
    cdict["dark-photon"]   = _count_coincidences_2arr(a_dark, b_photon, coinc_window)
    cdict["dark-dark"]     = _count_coincidences_2arr(a_dark, b_dark, coinc_window)
    return cdict

def _count_coincidences_2arr(times1, times2, window):
    i, j = 0, 0
    n_coinc = 0
    while i < len(times1) and j < len(times2):
        dt = times1[i] - times2[j]
        if abs(dt) <= window:
            n_coinc += 1
            i += 1
            j += 1
        elif dt < 0:
            i += 1
        else:
            j += 1
    return n_coinc

### Compute CHSH S-value from density matrices

In [6]:
def projector(theta):
    """ Projector onto cos(theta)|H> + sin(theta)|V> for 1 qubit. """
    state = np.cos(theta)*H + np.sin(theta)*V
    return ket2dm(state.unit())

def compute_expectation(rho_2q, theta_a, theta_b):
    A_plus = projector(theta_a)
    B_plus = projector(theta_b)

    A_minus = qeye(2) - A_plus
    B_minus = qeye(2) - B_plus

    P_pp = tensor(A_plus, B_plus)         # ++
    P_mm = tensor(A_minus, B_minus)       # --
    P_pm = tensor(A_plus, B_minus)        # +-
    P_mp = tensor(A_minus, B_plus)        # -+

    p_pp = (P_pp * rho_2q).tr().real
    p_mm = (P_mm * rho_2q).tr().real
    p_pm = (P_pm * rho_2q).tr().real
    p_mp = (P_mp * rho_2q).tr().real

    return (p_pp + p_mm) - (p_pm + p_mp)



In [15]:
# Parameters
PAIR_RATE   = 15000    # entangled pairs per second
DARK_RATE   = 1000     # dark counts/s
EFF_DET     = 0.10     # 10% detection efficiency
DEAD_TIME   = 4e-6     # 4 microseconds
T_RUN       = 30       # run for 30 seconds
COINC_WIN   = 1e-9     # 1 ns

results = {}

# total coincidence (any detection)
def _count_any_coinc(a_times, b_times):
    return _count_coincidences_2arr(a_times, b_times, COINC_WIN)

for state_name, rho_2q in all_states.items():
    # Generate entangled-pair arrival times
    pair_times = generate_pair_timestamps(PAIR_RATE, T_RUN)

    # Local detection for real photons
    alice_photon_t = detect_photons_from_pairs(pair_times, EFF_DET, DEAD_TIME, T_RUN)
    bob_photon_t   = detect_photons_from_pairs(pair_times, EFF_DET, DEAD_TIME, T_RUN)

    # Dark counts on each side
    alice_dark_t = generate_dark_counts(DARK_RATE, EFF_DET, DEAD_TIME, T_RUN)
    bob_dark_t   = generate_dark_counts(DARK_RATE, EFF_DET, DEAD_TIME, T_RUN)

    # Total detections
    alice_all = np.sort(np.concatenate([alice_photon_t, alice_dark_t]))
    bob_all   = np.sort(np.concatenate([bob_photon_t, bob_dark_t]))

    # Coincidence analysis 
    cdict = count_coincidences(alice_photon_t, alice_dark_t,
                                bob_photon_t, bob_dark_t,
                                coinc_window=COINC_WIN)
    
    total_coinc = _count_any_coinc(alice_all, bob_all)
    coincidence_rate = total_coinc / T_RUN

    # Detection rates
    alice_rate = len(alice_all) / T_RUN
    bob_rate   = len(bob_all)   / T_RUN

    # Fidelity
    F = fidelity(rho_2q, phi_plus)

    #S value
    E1 = compute_expectation(rho_2q, 0,       np.pi/8)
    E2 = compute_expectation(rho_2q, 0,       3*np.pi/8)
    E3 = compute_expectation(rho_2q, np.pi/4, np.pi/8)
    E4 = compute_expectation(rho_2q, np.pi/4, 3*np.pi/8)
    S = E1 - E2 + E3 + E4

    results[state_name] = {
        "Alice total rate (counts/s)": alice_rate,
        "Bob total rate (counts/s)":   bob_rate,

        "Alice photon rate": len(alice_photon_t)/T_RUN,
        "Alice dark rate":   len(alice_dark_t)/T_RUN,
        "Bob photon rate":   len(bob_photon_t)/T_RUN,
        "Bob dark rate":     len(bob_dark_t)/T_RUN,

        "Coincidences (ph-ph)":   cdict["photon-photon"],
        "Coincidences (ph-dark)": cdict["photon-dark"],
        "Coincidences (dark-ph)": cdict["dark-photon"],
        "Coincidences (dark-dark)": cdict["dark-dark"],

        "Total coincidences":       total_coinc,
        "Coincidence rate (counts/s)": coincidence_rate,

        "Fidelity vs. PhiPlus": float(F),
        "Bell S value": float(S)
    }

# Print results
for st, val in results.items():
    print(f"=== State: {st} ===")
    for k, v in val.items():
        print(f"  {k}: {v}")
    print()

=== State: Phi+ ===
  Alice total rate (counts/s): 1581.1
  Bob total rate (counts/s): 1594.0
  Alice photon rate: 1480.4
  Alice dark rate: 100.7
  Bob photon rate: 1495.9
  Bob dark rate: 98.1
  Coincidences (ph-ph): 4344
  Coincidences (ph-dark): 0
  Coincidences (dark-ph): 0
  Coincidences (dark-dark): 0
  Total coincidences: 4344
  Coincidence rate (counts/s): 144.8
  Fidelity vs. PhiPlus: 1.0000000052683549
  Bell S value: 2.828427124746189

=== State: rho ===
  Alice total rate (counts/s): 1582.2666666666667
  Bob total rate (counts/s): 1594.5333333333333
  Alice photon rate: 1484.7666666666667
  Alice dark rate: 97.5
  Bob photon rate: 1490.9333333333334
  Bob dark rate: 103.6
  Coincidences (ph-ph): 4511
  Coincidences (ph-dark): 0
  Coincidences (dark-ph): 0
  Coincidences (dark-dark): 0
  Total coincidences: 4511
  Coincidence rate (counts/s): 150.36666666666667
  Fidelity vs. PhiPlus: 0.8366600265340758
  Bell S value: 1.5556349186104041

=== State: phi ===
  Alice total ra

### Finding minimum S

In [11]:
phi_plus_ket = (tensor(H,H) + tensor(V,V)).unit()
phi_plus = ket2dm(phi_plus_ket)

# Werner State Constructor
def werner_state(p):
    identity_2qubit = qeye([2,2]) 
    identity_2qubit /= 4.0       

    return p * phi_plus + (1 - p) * identity_2qubit

def compute_chsh_s(rho_2q):
    a  = 0.0
    ap = np.pi/4
    b  = np.pi/8
    bp = 3*np.pi/8

    E_ab   = expectation_chsh(rho_2q, a,  b)
    E_abp  = expectation_chsh(rho_2q, a,  bp)
    E_apb  = expectation_chsh(rho_2q, ap, b)
    E_apbp = expectation_chsh(rho_2q, ap, bp)

    return E_ab - E_abp + E_apb + E_apbp

def expectation_chsh(rho_2q, theta_a, theta_b):
    A_plus = projector(theta_a)
    B_plus = projector(theta_b)
    A_minus = qeye(2) - A_plus
    B_minus = qeye(2) - B_plus

    P_pp = tensor(A_plus,  B_plus)   # ++
    P_mm = tensor(A_minus, B_minus)  # --
    P_pm = tensor(A_plus,  B_minus)  # +-
    P_mp = tensor(A_minus, B_plus)   # -+

    p_pp = (P_pp * rho_2q).tr().real
    p_mm = (P_mm * rho_2q).tr().real
    p_pm = (P_pm * rho_2q).tr().real
    p_mp = (P_mp * rho_2q).tr().real

    return (p_pp + p_mm) - (p_pm + p_mp)

In [14]:
p_values = np.linspace(0.70, 0.72, 100)  # 21 points from 0.70 to 0.72
data = []


for p in p_values:
    rhoW = werner_state(p)
    S_val = compute_chsh_s(rhoW)
    data.append((p, S_val))

print("p-value   S-value    Violates?")
threshold_found = False
for (p, S_val) in data:
    violates = (S_val > 2.0)
    print(f"{p:.4f}   {S_val:.6f}   {'Yes' if violates else 'No '}")
    if (not threshold_found) and violates:
        threshold_found = True
        print("^^ First p that yields S>2 ^^")

p-value   S-value    Violates?
0.7000   1.979899   No 
0.7002   1.980470   No 
0.7004   1.981042   No 
0.7006   1.981613   No 
0.7008   1.982185   No 
0.7010   1.982756   No 
0.7012   1.983327   No 
0.7014   1.983899   No 
0.7016   1.984470   No 
0.7018   1.985042   No 
0.7020   1.985613   No 
0.7022   1.986184   No 
0.7024   1.986756   No 
0.7026   1.987327   No 
0.7028   1.987899   No 
0.7030   1.988470   No 
0.7032   1.989041   No 
0.7034   1.989613   No 
0.7036   1.990184   No 
0.7038   1.990756   No 
0.7040   1.991327   No 
0.7042   1.991898   No 
0.7044   1.992470   No 
0.7046   1.993041   No 
0.7048   1.993613   No 
0.7051   1.994184   No 
0.7053   1.994755   No 
0.7055   1.995327   No 
0.7057   1.995898   No 
0.7059   1.996470   No 
0.7061   1.997041   No 
0.7063   1.997612   No 
0.7065   1.998184   No 
0.7067   1.998755   No 
0.7069   1.999327   No 
0.7071   1.999898   No 
0.7073   2.000469   Yes
^^^ First p that yields S>2 ^^^
0.7075   2.001041   Yes
0.7077   2.001612   Yes
0